# Res Net

In [1]:
import torch, time
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose
from torchvision import transforms  
from torch.utils.data import DataLoader


import matplotlib.pyplot as plt
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

import torch.nn as nn
from torch import optim
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"


In [2]:
print(device)


cpu


# Dataset


In [3]:
tf = Compose([
     transforms.ConvertImageDtype(torch.float),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.Pad(1)
 ])

class CustomCountriesDataset(Dataset):
    def __init__(self, df, transform=tf, target_transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = read_image(self.df.iloc[idx, 0])
        label = self.df.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
df = pd.read_csv("../output.csv")

X = df['images']
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)



In [5]:
#from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose
import numpy as np

In [6]:
train_dataset = CustomCountriesDataset(train_data, transform=tf)
test_dataset = CustomCountriesDataset(test_data, transform=tf)


In [7]:
bsz = 1
test_dataloader = DataLoader(test_dataset, batch_size=bsz, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=bsz, shuffle=False)

#print(next(iter(train_dataloader)))
"""
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features}")
print(f"Labels batch shape: {train_labels}")
img = train_features[0].squeeze()
label = train_labels[0]
image_np = img.numpy().transpose((1, 2, 0))
plt.imshow(image_np)
plt.show()
print(f"Label: {label}")"""



'\ntrain_features, train_labels = next(iter(train_dataloader))\nprint(f"Feature batch shape: {train_features}")\nprint(f"Labels batch shape: {train_labels}")\nimg = train_features[0].squeeze()\nlabel = train_labels[0]\nimage_np = img.numpy().transpose((1, 2, 0))\nplt.imshow(image_np)\nplt.show()\nprint(f"Label: {label}")'

In [8]:
class CNN(nn.Module):
    def __init__(self, in_channels=3):
        super(CNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels=6, kernel_size=3, stride=1, padding=1), # 224
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(6, 22, kernel_size=3, stride=1, padding=1), # 112
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(22, 76, kernel_size=3, stride=1, padding=1), # 56
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(76, 120, kernel_size=3, stride=1, padding=1), # 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(120, 200, kernel_size=3, stride=1, padding=1), # 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(200, 220, kernel_size=3, stride=1, padding=1), # 7
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 3

            nn.Flatten(),

            # FCL
            nn.Linear(220 * 3 * 3, 120),
            nn.Linear(120, 4),
            nn.Softmax(dim=1)
        )
        #pass
    def forward(self, x):
        return self.layers(x)

In [9]:
model = CNN(3).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.01)

model.train()
num_epochs = 1
for epoch in tqdm(range(num_epochs)):
    j = 0
    for i, (im, cl) in enumerate(train_dataloader):
        im = im.to(device)
        cl = cl.to(device)
        # forward pass
        pred = model(im)
        loss = criterion(pred, cl)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        j += 1

    print(j)

            
print("Finished!")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 